<a href="https://colab.research.google.com/github/calebahrens/chessengine/blob/main/Best_Chess_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import sys
king_move = 0
import time
import threading
game_moves = []
number_of_moves = 0

stop_event = threading.Event()
timer_thread = None

def start_timer():
    global timer_thread
    stop_event.clear()
    timer_thread = threading.Thread(target=countup_timer)
    timer_thread.start()

def stop_timer():
    if timer_thread and timer_thread.is_alive():
        stop_event.set()
        timer_thread.join()

def countup_timer():
    start_time = time.time()

    while not stop_event.is_set():
        elapsed = time.time() - start_time
        mins, secs = divmod(int(elapsed), 60)
        centiseconds = int((elapsed % 1) * 1000000000)
        timer = f'{mins:02d}:{secs:02d}.{centiseconds:02d}'
        time.sleep(0.000000001)

    print(f"\nMy move took: {timer}")
    print()


def score(board, turn):
    white_king_row, white_king_col = find_king(board, 'w')
    score = 0
    pieces = 0
    knight_squares = 0
    bishop_squares = 0
    queen_squares = 0
    rook_squares = 0
    bad_knight_squares = 0
    bad_bishop_squares = 0
    bad_queen_squares = 0
    bad_rook_squares = 0
    middlegame = False
    opening = False
    endgame = False
    developement = False
    pawn_take_queen = False
    rook_placed = False
    rook_placed_row = False
    rook_placed_col = False
    castled = False
    white_pieces = 0
    black_pieces = 0
    black_pawns = 0
    white_pawns = 0
    black_bishops = 0
    white_bishops = 0
    black_knights = 0
    white_knights = 0
    black_rooks = 0
    white_rooks = 0
    black_queens = 0
    white_queens = 0

    piece_values = {
        'p': 3, 'P': -3,
        'n': 9, 'N': -9,
        'b': 9, 'B': -9,
        'r': 15, 'R': -15,
        'q': 24, 'Q': -24,
        'k': 1000, 'K': -1000
    }

    for row in range(8):
        for col in range(8):
            if board[row][col] == 'k' and (row, col) == (7, 6)  and board[7][5] == 'r':
                castled = True
            if board[row][col] == 'k' and (row, col) == (7, 2)  and board[7][3] == 'r':
                castled = True
            if board[row][col] == 'k' and castled:
                score += 1.5
            if board[row][col] != '0':
                pieces += 1
            if board[row][col] in {'p', 'n', 'b', 'r', 'q', 'k'}:
                black_pieces += 1
            elif board[row][col] in {'P', 'N', 'B', 'R', 'Q', 'K'}:
                white_pieces += 1
            if board[row][col] == 'p':
                black_pawns += 1
            elif board[row][col] == 'P':
                white_pawns += 1
            elif board[row][col] == 'b':
                black_bishops += 1
            elif board[row][col] == 'B':
                white_bishops += 1
            elif board[row][col] == 'n':
                black_knights += 1
            elif board[row][col] == 'N':
                white_knights += 1
            elif board[row][col] == 'r':
                black_rooks += 1
            elif board[row][col] == 'r':
                black_rooks += 1
            elif board[row][col] == 'q':
                black_queens += 1
            elif board[row][col] == 'Q':
                white_queens += 1
            if (row, col) == (7, 7):
                if board[7][1] == '0' and board[7][2] == '0' and board[7][5] == '0' and board[7][6] == '0' and board[0][1] == '0' and board[0][2] == '0' and board[0][5] == '0' and board[0][6] == '0':
                    developement = True
                if pieces < 15:
                    endgame = True
                    opening = False
                    middlegame = False
                elif developement:
                    opening = False
                    middlegame = True
                    endgame = False
                else:
                    middlegame = False
                    opening = True
                    endgame = False

    for row in range(8):
        for col in range(8):
            piece = board[row][col]
            if piece in piece_values:
                score += piece_values[piece]
            if pieces == 2:
                print('Draw by insuffiecient material')
                sys.exit()
            if pieces == 3 and white_knights == 1:
                print('Draw by insuffiecient material')
                sys.exit()
            if pieces == 3 and black_knights == 1:
                print('Draw by insuffiecient material')
                sys.exit()
            if pieces == 4 and black_knights == 1 and white_knights == 1:
                print('Draw by insuffiecient material')
                sys.exit()
            if pieces == 4 and black_bishops == 1 and white_bishops == 1:
                print('Draw by insuffiecient material')
                sys.exit()
            if pieces == 3 and black_bishops == 1:
                print('Draw by insuffiecient material')
                sys.exit()
            if pieces == 3 and white_bishops == 1:
                print('Draw by insuffiecient material')
                sys.exit()
            if (row, col) in [(4, 3), (4, 4), (3, 3), (3, 4)]:
                if piece in {'n', 'b', 'r', 'q'}:
                    score += 0.4
                elif piece in {'N', 'B', 'R', 'Q'}:
                    score -= 0.4
                if piece == 'p':
                    score += 1.1
                elif piece == 'P':
                    score -= 1.1
            if piece == 'p' and (row, col) == (4, 3):
                if board[4][4] == 'p':
                  score += 0.3
            if piece == 'p' and (row, col) == (4, 4):
                if board[4][3] == 'p':
                  score += 0.3
            if piece == 'P' and (row, col) == (3, 3):
                if board[3][4] == 'P':
                  score -= 0.3
            if piece == 'P' and (row, col) == (3, 4):
                if board[3][3] == 'P':
                  score -= 0.3
            if (row, col) in [(5, 2), (4, 2), (5, 5), (4, 5), (5, 3), (5, 4)]:
                if piece == 'p':
                  score += 0.6
            if (row, col) in [(2, 2), (3, 2), (2, 5), (3, 5), (2, 3), (2, 4)]:
                if piece == 'P':
                  score -= 0.6
            if not endgame:
                if (row, col) == (5, 5) and piece == 'n' and board[4][4] == 'p':
                    score += 0.5
                if piece == 'p' and col in {2, 3, 4, 5}:
                    score -= row/10
                if piece == 'P' and col in {2, 3, 4, 5}:
                    score -= row/10
                if piece == 'p' and col not in {2, 3, 4, 5}:
                    score -= row/20
                if piece == 'P' and col not in {2, 3, 4, 5}:
                    score -= row/20
                if piece == 'b' and (row, col) == (3, 6) and board[2][5] == 'N' and board[0][3] == 'Q':
                    score += 0.5
                    if board[1][4] == 'B':
                      score -= 0.2
                if piece == 'B' and (row, col) == (4, 6) and board[5][5] == 'n' and board[7][3] == 'q':
                    score -= 0.5
                    if board[6][4] == 'b':
                      score -= 0.2
                if piece == 'n':
                    directions = [(1, 2), (1, -2), (-1, 2), (-1, -2), (2, 1), (2, -1), (-2, 1), (-2, -1)]
                    for direction in directions:
                        new_row = row + direction[0]
                        new_col = col + direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            knight_squares += 1
                    score += knight_squares/15
                elif piece == 'b':
                    directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
                    for direction in directions:
                        for i in range(1, 8):
                            new_row = row + i * direction[0]
                            new_col = col + i * direction[1]
                            if 0 <= new_row < 8 and 0 <= new_col < 8 and board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                bishop_squares += 1
                            else:
                                break
                    score += bishop_squares/15
                elif piece == 'r':
                    directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
                    for direction in directions:
                        for i in range(1, 8):
                            new_row = row + i * direction[0]
                            new_col = col + i * direction[1]
                            if 0 <= new_row < 8 and 0 <= new_col < 8 and board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                rook_squares += 1
                            else:
                                break
                    score += rook_squares/15
                elif piece == 'q':
                    directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
                    for direction in directions:
                        for i in range(1, 8):
                            new_row = row + i * direction[0]
                            new_col = col + i * direction[1]
                            if 0 <= new_row < 8 and 0 <= new_col < 8 and board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                queen_squares += 1
                            else:
                                break
                    score += queen_squares/15
                elif piece == 'N':
                    directions = [(1, 2), (1, -2), (-1, 2), (-1, -2), (2, 1), (2, -1), (-2, 1), (-2, -1)]
                    for direction in directions:
                        new_row = row + direction[0]
                        new_col = col + direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            bad_knight_squares += 1
                    score -= bad_knight_squares/15
                elif piece == 'B':
                    directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
                    for direction in directions:
                        for i in range(1, 8):
                            new_row = row + i * direction[0]
                            new_col = col + i * direction[1]
                            if 0 <= new_row < 8 and 0 <= new_col < 8 and board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                bad_bishop_squares += 1
                            else:
                                break
                    score -= bad_bishop_squares/15
                elif piece == 'R':
                    directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
                    for direction in directions:
                        for i in range(1, 8):
                            new_row = row + i * direction[0]
                            new_col = col + i * direction[1]
                            if 0 <= new_row < 8 and 0 <= new_col < 8 and board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                bad_rook_squares += 1
                            else:
                                break
                    score -= bad_rook_squares/15
                elif piece == 'Q':
                    directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
                    for direction in directions:
                        for i in range(1, 8):
                            new_row = row + i * direction[0]
                            new_col = col + i * direction[1]
                            if 0 <= new_row < 8 and 0 <= new_col < 8 and board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                bad_queen_squares += 1
                            else:
                                break
                    score -= bad_queen_squares/15
            if opening:
                if (row, col) in [(6, 4), (6, 2), (6, 3)] and piece == 'q':
                  score += 0.4
                if (row, col) in [(1, 4), (1, 2), (1, 3)] and piece == 'Q':
                  score -= 0.4
                if (row, col) == (7, 1) and piece == 'n':
                  score -= 0.7
                if (row, col) == (7, 6) and piece == 'n':
                  score -= 0.7
                if (row, col) == (7, 2) and piece == 'b':
                  score -= 0.7
                if (row, col) == (7, 5) and piece == 'b':
                  score -= 0.7
                if (row, col) == (0, 1) and piece == 'N':
                  score += 0.7
                if (row, col) == (0, 6) and piece == 'N':
                  score += 0.7
                if (row, col) == (0, 2) and piece == 'B':
                  score += 0.7
                if (row, col) == (0, 5) and piece == 'B':
                  score += 0.7
                if (row, col) in [(4, 3), (4, 4), (3, 3), (3, 4)]:
                    if piece == 'p':
                        score += 0.8
                    elif piece == 'P':
                        score -= 0.8
            if (col) == 0 or (col) == 7:
              if piece == 'n':
                  score -= 0.3
              if piece == 'N':
                  score += 0.3
            if (row) == 0 or (row) == 7:
              if piece == 'n':
                  score -= 0.3
              if piece == 'N':
                  score += 0.3
            if (row, col) == (4, 4):
              if piece == 'p':
                  score += 0.2
            if row in [5, 4, 3, 2] and piece == 'n':
              score += 0.2
            if row in [5, 4, 3, 2] and piece == 'N':
              score -= 0.2
            if row in [3, 2] and piece == 'n':
              score += 0.2
            if row in [4, 5] and piece == 'N':
              score -= 0.2
            if board[0][6] == 'K':
                if col in [5, 6, 7] and piece == 'q':
                    score += 0.7
                if col in [5, 6, 7] and piece == 'r':
                    score += 0.4
            if endgame:
                if piece == 'p':
                    score -= row^2/15
                if piece == 'P':
                    score -= row^2/15
                if row in {7, 0} or col in {7, 0}:
                    if piece == 'k':
                        score -= 0.5
                    if piece == 'K':
                        score += 0.5
                if white_pieces == 1:
                    if black_rooks == 2:
                        if piece == 'r':
                          if row + 1 == white_king_row or row -1 == white_king_row and not rook_placed_row and not rook_placed_col:
                              score += 3
                              rook_placed_row = True
                          if col + 1 == white_king_col or col - 1 == white_king_col and not rook_placed_row and not rook_placed_col:
                              score += 3
                              rook_placed_col = True
                          elif rook_placed_row:
                              if row == white_king_row:
                                  score += 3
                          elif rook_placed_col:
                              if col == white_king_col:
                                  score += 3
            VALUES = {
                'n': -9, 'N': 9, 'b': -9, 'B': 9, 'r': -15, 'R': 15,
                'q': -27, 'Q': 27, 'p': -3, 'P': 3
            }
            if piece in VALUES:
                  protected_color = 'w' if piece.isupper() else 'b'
                  threatened_color = 'b' if piece.isupper() else 'w'
                  threatened_by_enemy = is_protected(board, row, col, threatened_color)
                  not_protected_by_ally = is_protected(board, row, col, protected_color)
                  if threatened_by_enemy and not not_protected_by_ally:
                      if (piece.isupper() and turn == 'b') or (piece.islower() and turn == 'w'):
                          score += VALUES[piece]

            if piece == 'n':
              directions = [(1, 2), (1, -2), (-1, 2), (-1, -2), (2, 1), (2, -1), (-2, 1), (-2, -1)]
              for direction in directions:
                  new_row = row + direction[0]
                  new_col = col + direction[1]
                  if 0 <= new_row < 8 and 0 <= new_col < 8:
                      threatened_piece = board[new_row][new_col]
                      if threatened_piece in {'P', 'N', 'B', 'R'}:
                          score += 0.5
                      if threatened_piece in {'Q', 'K'}:
                          score += 0.7
            if turn == 'b' and piece in {'N', 'B', 'R', 'Q'}:
                directions = [(1, 1), (1, -1)]
                for direction in directions:
                    new_row = row + direction[0]
                    new_col = col + direction[1]
                    if 0 <= new_row < 8 and 0 <= new_col < 8:
                        if piece == 'Q' and board[new_row][new_col] == 'p':
                            if is_protected(board, row, col, 'w'):
                                score += 26
                        elif piece == 'R' and board[new_row][new_col] == 'p':
                            if is_protected(board, row, col, 'w'):
                                score += 14
                        elif piece in {'N', 'B'} and board[new_row][new_col] == 'p':
                            if is_protected(board, row, col, 'w'):
                                score += 8
            if turn == 'w' and piece in {'n', 'b', 'r', 'q'}:
                directions = [(-1, 1), (-1, -1)]
                for direction in directions:
                    new_row = row + direction[0]
                    new_col = col + direction[1]
                    if 0 <= new_row < 8 and 0 <= new_col < 8:
                        if piece == 'q' and board[new_row][new_col] == 'P' and pawn_take_queen == False and is_protected(board, row, col, 'b'):
                            score -= 26
                            pawn_take_queen = True
                        elif piece == 'r' and board[new_row][new_col] == 'P':
                            if is_protected(board, row, col, 'b'):
                                score -= 14
                        elif piece in {'n', 'b'} and board[new_row][new_col] == 'P':
                            if is_protected(board, row, col, 'b'):
                                score -= 8
            if turn == 'b' and piece in {'R', 'Q'}:
                directions = [(1, 2), (1, -2), (-1, 2), (-1, -2), (2, 1), (2, -1), (-2, 1), (-2, -1)]
                for direction in directions:
                    new_row = row + direction[0]
                    new_col = col + direction[1]
                    if 0 <= new_row < 8 and 0 <= new_col < 8:
                        if piece == 'Q' and board[new_row][new_col] == 'n':
                            if is_protected(board, row, col, 'w'):
                                score += 19
                        elif piece == 'R' and board[new_row][new_col] == 'n':
                            if is_protected(board, row, col, 'w'):
                                score += 7
            if turn == 'w' and piece in {'r', 'q'}:
                  directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
                  for direction in directions:
                      for i in range(1, 8):
                          new_row = row + i * direction[0]
                          new_col = col + i * direction[1]
                          if 0 <= new_row < 8 and 0 <= new_col < 8:
                              if piece == 'q' and board[new_row][new_col] == 'B':
                                  if is_protected(board, row, col, 'b'):
                                      score -= 19
                              elif piece == 'r' and board[new_row][new_col] == 'B':
                                  if is_protected(board, row, col, 'b'):
                                      score -= 7
                              if board[new_row][new_col] != '0':
                                  break
                          else:
                              break
            if turn == 'b' and piece in {'R', 'Q'}:
                directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
                for direction in directions:
                    for i in range(1, 8):
                        new_row = row + i * direction[0]
                        new_col = col + i * direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if piece == 'Q' and board[new_row][new_col] == 'b':
                                if is_protected(board, row, col, 'w'):
                                    score -= 21.6
                            elif piece == 'R' and board[new_row][new_col] == 'b':
                                if is_protected(board, row, col, 'w'):
                                    score -= 13
                            if board[new_row][new_col] != '0':
                                break
                        else:
                            break
            if turn == 'w' and piece == 'q':
                    directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
                    for direction in directions:
                        for i in range(1, 8):
                            new_row = row + i * direction[0]
                            new_col = col + i * direction[1]
                            if 0 <= new_row < 8 and 0 <= new_col < 8:
                                if board[new_row][new_col] == 'R':
                                    if is_protected(board, row, col, 'b'):
                                        score -= 13
                                if board[new_row][new_col] != '0':
                                    break
                            else:
                                break
            if turn == 'b' and piece == 'Q':
                directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
                for direction in directions:
                    for i in range(1, 8):
                        new_row = row + i * direction[0]
                        new_col = col + i * direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] == 'r':
                                if is_protected(board, row, col, 'w'):
                                    score += 13
                            if board[new_row][new_col] != '0':
                                break
                        else:
                            break


    starting_penalty_positions = {
        (7, 1): -0.5, (7, 6): -0.5,
        (7, 2): -0.5, (7, 5): -0.5,
        (0, 1): 0.5, (0, 6): 0.5,
        (0, 2): 0.5, (0, 5): 0.5
    }

    for (row, col), adjustment in starting_penalty_positions.items():
        piece = board[row][col]
        if piece.lower() in {'n', 'b', 'N', 'B'}:
            score += adjustment if piece.islower() else -adjustment

    return score

def pos_to_indices(pos):
    col = ord(pos[0]) - ord('a')
    row = 8 - int(pos[1])
    return row, col

def indices_to_pos(row, col):
    col_pos = chr(col + ord('a'))
    row_pos = str(8 - row)
    return col_pos + row_pos

def indices_to_pos_col(col):
    col_pos = chr(col + ord('a'))
    return col_pos

def indices_to_pos_row(row):
    row_pos = str(8 - row)
    return row_pos

def find_king(board, king_color):
    king_symbol = 'K' if king_color == 'w' else 'k'
    for row in range(8):
        for col in range(8):
            if board[row][col] == king_symbol:
                return row, col
    print('here')
    print_board(board)
    return None

def is_king_in_check(board, king_row, king_col, player):
    opponent = 'b' if player == 'w' else 'w'

    for row in range(8):
        for col in range(8):
            piece = board[row][col]
            if piece != '0' and \
                ((piece.islower() and player == 'w') or (piece.isupper() and player == 'b')):

                if piece.lower() == 'b' or piece.lower() == 'q':
                    directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
                    for direction in directions:
                        for i in range(1, 8):
                            new_row = row + i * direction[0]
                            new_col = col + i * direction[1]
                            if 0 <= new_row < 8 and 0 <= new_col < 8:
                                if new_row == king_row and new_col == king_col:
                                    return True
                                if board[new_row][new_col] != '0':
                                    break
                            else:
                                break

                if piece.lower() == 'r' or piece.lower() == 'q':
                    directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
                    for direction in directions:
                        for i in range(1, 8):
                            new_row = row + i * direction[0]
                            new_col = col + i * direction[1]
                            if 0 <= new_row < 8 and 0 <= new_col < 8:
                                if new_row == king_row and new_col == king_col:
                                    return True
                                if board[new_row][new_col] != '0':
                                    break
                            else:
                                break

                elif piece.lower() == 'n':
                    knight_moves = [(1, 2), (1, -2), (-1, 2), (-1, -2),
                                    (2, 1), (2, -1), (-2, 1), (-2, -1)]
                    for move in knight_moves:
                        new_row = row + move[0]
                        new_col = col + move[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if new_row == king_row and new_col == king_col:
                                return True

                elif piece.lower() == 'p':
                    if player == 'b':
                        pawn_attacks = [(1, 1), (1, -1)]
                    else:
                        pawn_attacks = [(-1, 1), (-1, -1)]

                    for attack in pawn_attacks:
                        new_row = row + attack[0]
                        new_col = col + attack[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if new_row == king_row and new_col == king_col:
                                return True
                elif piece.lower() == 'k':
                    directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
                    for move in directions:
                        new_row = row + move[0]
                        new_col = col + move[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if new_row == king_row and new_col == king_col:
                                return True

    return False

def is_protected(board, row, col, player):
    opponent = 'b' if player == 'w' else 'w'
    if player == 'b':
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
        for direction in directions:
            for i in range(1, 8):
                new_row = row + i * direction[0]
                new_col = col + i * direction[1]
                if 0 <= new_row < 8 and 0 <= new_col < 8:
                    if board[new_row][new_col] in {'r', 'q'}:
                        if board[row][col] == 'n' and (row, col) == (3, 4) and player == 'w':
                            print('yay!')
                        return True
                    if board[new_row][new_col] != '0':
                        break
                else:
                    break

        directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
        for direction in directions:
            for i in range(1, 8):
                new_row = row + i * direction[0]
                new_col = col + i * direction[1]
                if 0 <= new_row < 8 and 0 <= new_col < 8:
                    if board[new_row][new_col] in {'b', 'q'}:
                        if board[row][col] == 'n' and (row, col) == (3, 4) and player == 'w':
                            print('yay!')
                        return True
                    if board[new_row][new_col] != '0':
                        break
                else:
                    break

        directions = [(1, 2), (1, -2), (-1, 2), (-1, -2), (2, 1), (2, -1), (-2, 1), (-2, -1)]
        for direction in directions:
            new_row = row + direction[0]
            new_col = col + direction[1]
            if 0 <= new_row < 8 and 0 <= new_col < 8:
                if board[new_row][new_col] == 'n':
                    if board[row][col] == 'n' and (row, col) == (3, 4) and player == 'w':
                        print('yay!')
                    return True

        pawn_attacks = [(1, 1), (1, -1)]
        for attack in pawn_attacks:
            new_row = row + attack[0]
            new_col = col + attack[1]
            if 0 <= new_row < 8 and 0 <= new_col < 8:
                if board[new_row][new_col] == 'p':
                    if board[row][col] == 'n' and (row, col) == (3, 4) and player == 'w':
                        print('yay!')
                    return True
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
        for direction in directions:
            new_row = row + direction[0]
            new_col = col + direction[1]
            if 0 <= new_row < 8 and 0 <= new_col < 8:
                if board[new_row][new_col] == 'k':
                    if board[row][col] == 'n' and (row, col) == (3, 4) and player == 'w':
                        print('yay!')
                    return True

    elif player == 'w':
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
        for direction in directions:
            for i in range(1, 8):
                new_row = row + i * direction[0]
                new_col = col + i * direction[1]
                if 0 <= new_row < 8 and 0 <= new_col < 8:
                    if board[new_row][new_col] in {'R', 'Q'}:
                        return True
                    if board[new_row][new_col] != '0':
                        break
                else:
                    break

        directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
        for direction in directions:
            for i in range(1, 8):
                new_row = row + i * direction[0]
                new_col = col + i * direction[1]
                if 0 <= new_row < 8 and 0 <= new_col < 8:
                    if board[new_row][new_col] in {'B', 'Q'}:
                        return True
                    if board[new_row][new_col] != '0':
                        break
                else:
                    break

        directions = [(1, 2), (1, -2), (-1, 2), (-1, -2), (2, 1), (2, -1), (-2, 1), (-2, -1)]
        for direction in directions:
            new_row = row + direction[0]
            new_col = col + direction[1]
            if 0 <= new_row < 8 and 0 <= new_col < 8:
                if board[new_row][new_col] == 'N':
                    return True

        pawn_attacks = [(-1, 1), (-1, -1)]
        for attack in pawn_attacks:
            new_row = row + attack[0]
            new_col = col + attack[1]
            if 0 <= new_row < 8 and 0 <= new_col < 8:
                if board[new_row][new_col] == 'P':
                    return True
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
        for direction in directions:
            new_row = row + direction[0]
            new_col = col + direction[1]
            if 0 <= new_row < 8 and 0 <= new_col < 8:
                if board[new_row][new_col] == 'K':
                    return True

    return False

def players_turn(board):
    from_square = input("From what square: ").strip().lower()
    if from_square == '0-0':
      move_played = '0-0'
      board[0][4] = '0'
      board[0][6] = 'K'
      board[0][7] = '0'
      board[0][5] = 'R'
      print_board(board)
      print()
    elif from_square == '0-0-0':
      move_played = '0-0-0'
      board[0][4] = '0'
      board[0][2] = 'K'
      board[0][0] = '0'
      board[0][3] = 'R'
      print_board(board)
      print()
    else:
        to_square = input("To what square: ").strip().lower()
        from_row, from_col = pos_to_indices(from_square)
        to_row, to_col = pos_to_indices(to_square)
        s = board[to_row][to_col]
        if board[from_row][from_col] != '0' and s != 'P' and s != 'R' and s != 'N' and s != 'B' and s != 'K' and s != 'Q':
            piece = board[from_row][from_col]
            captured = board[to_row][to_row]
            if piece == 'P' and to_row == 7:
                board[from_row][from_col] = '0'
                board[to_row][to_col] = 'Q'
            else:
                board[from_row][from_col] = '0'
                board[to_row][to_col] = piece
            black_king_row, black_king_col = find_king(board, 'b')
            if is_king_in_check(board, black_king_row, black_king_col, 'b'):
                if piece != 'P':
                    if captured in {'p', 'n', 'b', 'r', 'q'}:
                      move_played = piece.upper() + 'x' + indices_to_pos(to_row, to_col) + '+'
                    else:
                      move_played = piece.upper() + indices_to_pos(to_row, to_col) + '+'
                else:
                    if captured in {'p', 'n', 'b', 'r', 'q'}:
                      move_played = indices_to_pos_col(from_col) + 'x' + indices_to_pos(to_row, to_col) + '+'
                    else:
                      move_played = indices_to_pos(to_row, to_col) + '+'
            else:
                if piece != 'P':
                    if captured in {'p', 'n', 'b', 'r', 'q'}:
                      move_played = piece.upper() + 'x' + indices_to_pos(to_row, to_col)
                    else:
                      move_played = piece.upper() + indices_to_pos(to_row, to_col)
                else:
                    if captured in {'p', 'n', 'b', 'r', 'q'}:
                      move_played = indices_to_pos_col(from_col) + 'x' + indices_to_pos(to_row, to_col)
                    else:
                      move_played = indices_to_pos(to_row, to_col)
            print_board(board)
            print()
            start_timer()
        else:
            print("Invalid move. Try again.")
            players_turn(board)
    game_moves.append(move_played)

def initialize_board():
    if input('hello') == 'hey':
        board = [[' ' for _ in range(8)] for _ in range(8)]
        board[0] = ['0'] * 8
        board[1] = ['0'] * 8
        board[2] = ['0', '0', '0', '0', '0', '0', '0', 'p']
        board[3] = ['0', '0', '0', '0', 'K', '0', '0', '0']
        board[4] = ['0'] * 8
        board[5] = ['0'] * 8
        board[6] = ['0'] * 8
        board[7] = ['r', '0', '0', '0', 'k', '0', '0', 'r']
        print(board)
        return board
    else:
        board = [[' ' for _ in range(8)] for _ in range(8)]
        board[0] = ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']
        board[1] = ['P'] * 8
        board[2] = ['0'] * 8
        board[3] = ['0'] * 8
        board[4] = ['0'] * 8
        board[5] = ['0'] * 8
        board[6] = ['p'] * 8
        board[7] = ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r']
        print_board(board)
        return board

def print_board(board):
    for row in board:
        print(' '.join(str(cell) for cell in row))


def pick_random_square(board):
    while True:
        row = random.randint(0, 7)
        col = random.randint(0, 7)
        s = board[row][col]
        if s == 'p' or s == 'r' or s == 'n' or s == 'b' or s == 'k' or s == 'q':
            return row, col

board = initialize_board()

def best_move(board):
    global king_move
    global number_of_moves
    black_king_row, black_king_col = find_king(board, 'b')
    checkmate = False
    checkmate2 = False
    bad_checkmate = False
    stalemate = False
    promotion = False
    previous_score = -6000
    best_moves = []
    rows = list(range(8))
    random.shuffle(rows)
    cols = list(range(8))
    random.shuffle(cols)
    good_moves = [(6, 4, 4, 4, 'p', '0'), (7, 6, 5, 5, 'n', '0'), (7, 6, 5, 5, 'n', '0'), (5, 3, 1, 7, 'b', 'P'), (7, 5, 4, 2, 'b', '0')]
    if board == [['R', '0', 'B', 'Q', 'K', '0', 'N', 'R'], ['P', 'P', 'P', 'P', '0', 'P', 'P', 'P'], ['0', '0', 'N', '0', '0', '0', '0', '0'], ['0', '0', 'B', '0', 'P', '0', '0', '0'], ['0', '0', 'b', '0', 'p', '0', '0', '0'], ['0', '0', '0', '0', '0', 'n', '0', '0'], ['p', 'p', 'p', 'p', '0', 'p', 'p', 'p'], ['r', 'n', 'b', 'q', 'k', '0', '0', 'r']]:
        best_moves = [(6, 2, 5, 2, 'p')]
        previous_score = score(board, 'w')
    elif board == [['R', '0', 'B', 'Q', 'K', '0', '0', 'R'], ['P', 'P', 'P', 'P', '0', 'P', 'P', 'P'], ['0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0'], ['0', 'N', 'b', 'p', 'N', '0', '0', '0'], ['0', '0', '0', '0', '0', 'n', '0', '0'], ['p', 'p', '0', '0', '0', 'p', 'p', 'p'], ['r', 'n', '0', 'q', 'k', '0', '0', 'r']]:
        best_moves = [(4, 2, 1, 5, 'b')]
        previous_score = score(board, 'w')
    else:
        for row in rows:
            for col in cols:
                piece = board[row][col]
                if piece == 'p':
                    directions = [1, 2, 3, 4]
                    random.shuffle(directions)
                    for direction in directions:
                        if direction == 1 and row == 6 and board[row-2][col] == '0' and board[row-1][col] == '0':
                            print(indices_to_pos(row-2, col))
                            board[row][col] = '0'
                            board[row-2][col] = 'p'
                            if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                    white_king_row, white_king_col = find_king(board, 'w')
                                    if is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                        checkmate = True
                                    else:
                                        stalemate = True
                                elif best_row == best_col == target_row == target_col == best_piece == captured == '2':
                                    bad_checkmate = True
                                if not checkmate and not bad_checkmate and not stalemate:
                                    if best_piece != 'P':
                                      if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                        print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                      else:
                                        print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                    else:
                                        if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                          print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                        else:
                                          print(indices_to_pos(target_row, target_col))
                                    board[best_row][best_col] = '0'
                                    board[target_row][target_col] = best_piece
                                    if target_row == 7 and best_piece == 'P':
                                        board[target_row][target_col] = 'Q'
                                    best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                    if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                        checkmate2 = True
                                    if not checkmate2:
                                        if best_piece2 != 'p':
                                            if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                              print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                            else:
                                              print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                        else:
                                            if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                              print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                            else:
                                              print(indices_to_pos(target_row2, target_col2))
                                        board[best_row2][best_col2] = '0'
                                        board[target_row2][target_col2] = best_piece2
                                        if target_row2 == 0 and best_piece2 == 'p':
                                            board[target_row2][target_col2] = 'q'
                                        current_score = score(board, 'w')
                                        for move in good_moves:
                                            if (row, col, row-2, col, 'p', '0') == move:
                                                current_score += 0.5
                                        print(current_score)
                                        print()
                                        board[best_row2][best_col2] = best_piece2
                                        board[target_row2][target_col2] = captured2
                                        board[best_row][best_col] = best_piece
                                        board[target_row][target_col] = captured
                                        if current_score > previous_score:
                                            previous_score = current_score
                                            best_moves = [(row, col, row-2, col, 'p')]
                                        elif current_score == previous_score:
                                            best_moves.append((row, col, row-2, col, 'p'))
                                elif checkmate:
                                    print_board(board)
                                    print('CHECKMATE! you lose')
                                    sys.exit()
                                elif bad_checkmate:
                                    current_score = -1000
                                    if current_score > previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, row-2, col, 'p')]
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, row-2, col, 'p'))
                            board[row][col] = 'p'
                            board[row-2][col] = '0'
                            checkmate = False
                            checkmate2 = False
                            bad_checkmate = False
                            stalemate = False

                        elif direction == 2 and row > 0 and board[row-1][col] == '0':
                            print(indices_to_pos(row-1, col))
                            board[row][col] = '0'
                            board[row-1][col] = 'p'
                            if row-1 == 0:
                                board[row-1][col] = 'q'
                                promotion = True
                                print_board(board)
                            if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                    white_king_row, white_king_col = find_king(board, 'w')
                                    if is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                        checkmate = True
                                    else:
                                        stalemate = True
                                if best_row == best_col == target_row == target_col == best_piece == captured == '2':
                                  bad_checkmate = True
                                if not checkmate and not bad_checkmate and not stalemate:
                                    if best_piece != 'P':
                                      if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                        print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                      else:
                                        print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                    else:
                                        if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                          print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                        else:
                                          print(indices_to_pos(target_row, target_col))
                                    board[best_row][best_col] = '0'
                                    board[target_row][target_col] = best_piece
                                    if target_row == 7 and best_piece == 'P':
                                        board[target_row][target_col] = 'Q'
                                    best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                    if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                        checkmate2 = True
                                    if not checkmate2:
                                        if best_piece2 != 'p':
                                            if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                              print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                            else:
                                              print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                        else:
                                            if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                              print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                            else:
                                              print(indices_to_pos(target_row2, target_col2))
                                        board[best_row2][best_col2] = '0'
                                        board[target_row2][target_col2] = best_piece2
                                        if target_row2 == 0 and best_piece2 == 'p':
                                            board[target_row2][target_col2] = 'q'
                                        current_score = score(board, 'w')
                                        for move in good_moves:
                                            if (row, col, row-1, col, 'p', '0') == move:
                                                current_score += 0.5
                                        print(current_score)
                                        print()
                                        board[best_row2][best_col2] = best_piece2
                                        board[target_row2][target_col2] = captured2
                                        board[best_row][best_col] = best_piece
                                        board[target_row][target_col] = captured
                                        if current_score > previous_score:
                                            previous_score = current_score
                                            if promotion:
                                                best_moves = [(row, col, row-1, col, 'q')]
                                                promotion = False
                                            else:
                                                best_moves = [(row, col, row-1, col, 'p')]
                                        elif current_score == previous_score:
                                            if promotion:
                                                best_moves.append((row, col, row-1, col, 'q'))
                                                promotion = False
                                            else:
                                                best_moves.append((row, col, row-1, col, 'p'))
                                elif checkmate:
                                    print_board(board)
                                    print('CHECKMATE! you lose')
                                    sys.exit()
                                elif bad_checkmate:
                                    current_score = -1000
                                    if current_score > previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, row-1, col, 'p')]
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, row-1, col, 'p'))
                            board[row][col] = 'p'
                            board[row-1][col] = '0'
                            checkmate = False
                            checkmate2 = False
                            bad_checkmate = False
                            stalemate = False

                        elif direction == 3 and row > 0 and col > 0 and board[row-1][col-1] in {'P', 'N', 'B', 'R', 'Q'}:
                            captured_piece = board[row-1][col-1]
                            print(indices_to_pos_col(col), 'x', indices_to_pos(row-1, col-1))
                            board[row][col] = '0'
                            board[row-1][col-1] = 'p'
                            if row-1 == 0:
                                board[row-1][col-1] = 'q'
                                promotion = True
                            if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                    white_king_row, white_king_col = find_king(board, 'w')
                                    if is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                        checkmate = True
                                    else:
                                        stalemate = True
                                if best_row == best_col == target_row == target_col == best_piece == captured == '2':
                                    bad_checkmate = True
                                if not checkmate and not bad_checkmate and not stalemate:
                                    if best_piece != 'P':
                                      if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                        print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                      else:
                                        print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                    else:
                                        if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                          print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                        else:
                                          print(indices_to_pos(target_row, target_col))
                                    board[best_row][best_col] = '0'
                                    board[target_row][target_col] = best_piece
                                    if target_row == 7 and best_piece == 'P':
                                        board[target_row][target_col] = 'Q'
                                    best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                    if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                        checkmate2 = True
                                    if not checkmate2:
                                        if best_piece2 != 'p':
                                            if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                              print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                            else:
                                              print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                        else:
                                            if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                              print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                            else:
                                              print(indices_to_pos(target_row2, target_col2))
                                        board[best_row2][best_col2] = '0'
                                        board[target_row2][target_col2] = best_piece2
                                        if target_row2 == 0 and best_piece2 == 'p':
                                            board[target_row2][target_col2] = 'q'
                                        current_score = score(board, 'w')
                                        for move in good_moves:
                                            if (row, col, row-1, col-1, 'p', captured_piece) == move:
                                                current_score += 0.5
                                        print(current_score)
                                        print()
                                        board[best_row2][best_col2] = best_piece2
                                        board[target_row2][target_col2] = captured2
                                        board[best_row][best_col] = best_piece
                                        board[target_row][target_col] = captured
                                        if current_score > previous_score:
                                            previous_score = current_score
                                            if promotion:
                                                best_moves = [(row, col, row-1, col-1, 'q')]
                                                promotion = False
                                            else:
                                                best_moves = [(row, col, row-1, col-1, 'p')]
                                        elif current_score == previous_score:
                                            if promotion:
                                                best_moves.append((row, col, row-1, col-1, 'q'))
                                                promotion = False
                                            else:
                                                best_moves.append((row, col, row-1, col-1, 'p'))
                                elif checkmate:
                                    print_board(board)
                                    print('CHECKMATE! you lose')
                                    sys.exit()
                                elif bad_checkmate:
                                    current_score = -1000
                                    if current_score > previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, row-1, col-1, 'p')]
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, row-1, col-1, 'p'))
                            board[row][col] = 'p'
                            board[row-1][col-1] = captured_piece
                            checkmate = False
                            checkmate2 = False
                            bad_checkmate = False
                            stalemate = False

                        elif direction == 4 and row > 0 and col < 7 and board[row-1][col+1] in {'P', 'N', 'B', 'R', 'Q'}:
                            captured_piece = board[row-1][col+1]
                            print(indices_to_pos_col(col), 'x', indices_to_pos(row-1, col+1))
                            board[row][col] = '0'
                            board[row-1][col+1] = 'p'
                            if row-1 == 0:
                                board[row-1][col+1] = 'q'
                                promotion = True
                            if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                    white_king_row, white_king_col = find_king(board, 'w')
                                    if is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                        checkmate = True
                                    else:
                                        stalemate = True
                                if best_row == best_col == target_row == target_col == best_piece == captured == '2':
                                    bad_checkmate = True
                                if not checkmate and not bad_checkmate and not stalemate:
                                    if best_piece != 'P':
                                      if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                        print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                      else:
                                        print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                    else:
                                        if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                          print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                        else:
                                          print(indices_to_pos(target_row, target_col))
                                    board[best_row][best_col] = '0'
                                    board[target_row][target_col] = best_piece
                                    if target_row == 7 and best_piece == 'P':
                                        board[target_row][target_col] = 'Q'
                                    best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                    if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                        checkmate2 = True
                                    if not checkmate2:
                                        if best_piece2 != 'p':
                                            if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                              print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                            else:
                                              print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                        else:
                                            if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                              print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                            else:
                                              print(indices_to_pos(target_row2, target_col2))
                                        board[best_row2][best_col2] = '0'
                                        board[target_row2][target_col2] = best_piece2
                                        if target_row2 == 0 and best_piece2 == 'p':
                                            board[target_row2][target_col2] = 'q'
                                        current_score = score(board, 'w')
                                        for move in good_moves:
                                            if (row, col, row-1, col+1, 'p', captured_piece) == move:
                                                current_score += 0.5
                                        print(current_score)
                                        print()
                                        board[best_row2][best_col2] = best_piece2
                                        board[target_row2][target_col2] = captured2
                                        board[best_row][best_col] = best_piece
                                        board[target_row][target_col] = captured
                                        if current_score > previous_score:
                                            previous_score = current_score
                                            if promotion:
                                                best_moves = [(row, col, row-1, col+1, 'q')]
                                                promotion = False
                                            else:
                                                best_moves = [(row, col, row-1, col+1, 'p')]
                                        elif current_score == previous_score:
                                            if promotion:
                                                best_moves.append((row, col, row-1, col+1, 'q'))
                                                promotion = False
                                            else:
                                                best_moves.append((row, col, row-1, col+1, 'p'))
                                elif checkmate:
                                    print_board(board)
                                    print('CHECKMATE! you lose')
                                    sys.exit()
                                elif bad_checkmate:
                                    current_score = -1000
                                    if current_score > previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, row-1, col+1, 'p')]
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, row-1, col+1, 'p'))
                            board[row][col] = 'p'
                            board[row-1][col+1] = captured_piece
                            checkmate = False
                            checkmate2 = False
                            bad_checkmate = False
                            stalemate = False


                elif piece == 'n':
                    directions = [(1, 2), (1, -2), (-1, 2), (-1, -2), (2, 1), (2, -1), (-2, 1), (-2, -1)]
                    random.shuffle(directions)
                    for direction in directions:
                        new_row = row + direction[0]
                        new_col = col + direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                captured_piece = board[new_row][new_col]
                                if board[new_row][new_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                  print('N' + 'x' + indices_to_pos(new_row, new_col))
                                else:
                                  print('N' + indices_to_pos(new_row, new_col))
                                board[row][col] = '0'
                                board[new_row][new_col] = 'n'
                                if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                    best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                    if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                        white_king_row, white_king_col = find_king(board, 'w')
                                        if is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                            checkmate = True
                                        else:
                                            stalemate = True
                                    if best_row == best_col == target_row == target_col == best_piece == captured == '2':
                                        bad_checkmate = True
                                    if not checkmate and not bad_checkmate and not stalemate:
                                        if best_piece != 'P':
                                          if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                            print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                          else:
                                            print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                        else:
                                          if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                            print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                          else:
                                            print(indices_to_pos(target_row, target_col))
                                        board[best_row][best_col] = '0'
                                        board[target_row][target_col] = best_piece
                                        if target_row == 7 and best_piece == 'P':
                                            board[target_row][target_col] = 'Q'
                                        best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                        if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                            checkmate2 = True
                                        if not checkmate2:
                                            if best_piece2 != 'p':
                                                if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                  print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                                else:
                                                  print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                            else:
                                                if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                  print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                                else:
                                                  print(indices_to_pos(target_row2, target_col2))
                                            board[best_row2][best_col2] = '0'
                                            board[target_row2][target_col2] = best_piece2
                                            if target_row2 == 0 and best_piece2 == 'p':
                                                board[target_row2][target_col2] = 'q'
                                            current_score = score(board, 'w')
                                            for move in good_moves:
                                                if (row, col, new_row, new_col, 'n', captured_piece) == move:
                                                    current_score += 0.5
                                            print(current_score)
                                            print()
                                            board[best_row2][best_col2] = best_piece2
                                            board[target_row2][target_col2] = captured2
                                            board[best_row][best_col] = best_piece
                                            board[target_row][target_col] = captured
                                            if current_score > previous_score:
                                                previous_score = current_score
                                                best_moves = [(row, col, new_row, new_col, 'n')]
                                            elif current_score == previous_score:
                                                best_moves.append((row, col, new_row, new_col, 'n'))
                                    elif checkmate:
                                        print_board(board)
                                        print('CHECKMATE! you lose')
                                        sys.exit()
                                    elif bad_checkmate:
                                        current_score = -1000
                                        if current_score > previous_score:
                                            previous_score = current_score
                                            best_moves = [(row, col, new_row, new_col, 'n')]
                                        elif current_score == previous_score:
                                            best_moves.append((row, col, new_row, new_col, 'n'))
                                board[row][col] = 'n'
                                board[new_row][new_col] = captured_piece
                                checkmate = False
                                checkmate2 = False
                                bad_checkmate = False
                                stalemate = False


                elif piece == 'b':
                    directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
                    random.shuffle(directions)
                    for direction in directions:
                        for i in range(1, 8):
                            new_row = row + i * direction[0]
                            new_col = col + i * direction[1]
                            if 0 <= new_row < 8 and 0 <= new_col < 8:
                                if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                    captured_piece = board[new_row][new_col]
                                    if board[new_row][new_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                      print('B' + 'x' + indices_to_pos(new_row, new_col))
                                    else:
                                      print('B' + indices_to_pos(new_row, new_col))
                                    board[row][col] = '0'
                                    board[new_row][new_col] = 'b'
                                    if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                        best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                        if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                            white_king_row, white_king_col = find_king(board, 'w')
                                            if is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                                checkmate = True
                                            else:
                                                stalemate = True
                                        if best_row == best_col == target_row == target_col == best_piece == captured == '2':
                                            bad_checkmate = True
                                        if not checkmate and not bad_checkmate and not stalemate:
                                            if best_piece != 'P':
                                              if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                              else:
                                                print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                            else:
                                              if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                              else:
                                                print(indices_to_pos(target_row, target_col))
                                            board[best_row][best_col] = '0'
                                            board[target_row][target_col] = best_piece
                                            if target_row == 7 and best_piece == 'P':
                                                board[target_row][target_col] = 'Q'
                                            best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                            if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                                checkmate2 = True
                                            if not checkmate2:
                                                if best_piece2 != 'p':
                                                  if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                    print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                                  else:
                                                    print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                                else:
                                                  if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                    print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                                  else:
                                                    print(indices_to_pos(target_row2, target_col2))
                                                board[best_row2][best_col2] = '0'
                                                board[target_row2][target_col2] = best_piece2
                                                if target_row2 == 0 and best_piece2 == 'p':
                                                    board[target_row2][target_col2] = 'q'
                                                current_score = score(board, 'w')
                                                for move in good_moves:
                                                    if (row, col, new_row, new_col, 'b', captured_piece) == move:
                                                        current_score += 0.5
                                                print(current_score)
                                                print()
                                                board[best_row2][best_col2] = best_piece2
                                                board[target_row2][target_col2] = captured2
                                                board[best_row][best_col] = best_piece
                                                board[target_row][target_col] = captured
                                                if current_score > previous_score:
                                                    previous_score = current_score
                                                    best_moves = [(row, col, new_row, new_col, 'b')]
                                                elif current_score == previous_score:
                                                    best_moves.append((row, col, new_row, new_col, 'b'))
                                        elif checkmate:
                                            print_board(board)
                                            print('CHECKMATE! you lose')
                                            sys.exit()
                                        elif bad_checkmate:
                                            current_score = -1000
                                            if current_score > previous_score:
                                                previous_score = current_score
                                                best_moves = [(row, col, new_row, new_col, 'b')]
                                            elif current_score == previous_score:
                                                best_moves.append((row, col, new_row, new_col, 'b'))
                                    board[row][col] = 'b'
                                    board[new_row][new_col] = captured_piece
                                    checkmate = False
                                    checkmate2 = False
                                    bad_checkmate = False
                                    stalemate = False

                                else:
                                    break
                                if board[new_row][new_col] == 'P' or board[new_row][new_col] == 'N' or board[new_row][new_col] == 'B' or board[new_row][new_col] == 'R' or board[new_row][new_col] == 'Q':
                                    break
                            else:
                                break

                elif piece == 'r':
                    directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
                    random.shuffle(directions)
                    for direction in directions:
                        for i in range(1, 8):
                            new_row = row + i * direction[0]
                            new_col = col + i * direction[1]
                            if 0 <= new_row < 8 and 0 <= new_col < 8:
                                if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                    captured_piece = board[new_row][new_col]
                                    if board[new_row][new_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                      print('R' + 'x' + indices_to_pos(new_row, new_col))
                                    else:
                                      print('R' + indices_to_pos(new_row, new_col))
                                    board[row][col] = '0'
                                    board[new_row][new_col] = 'r'
                                    if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                        best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                        if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                            white_king_row, white_king_col = find_king(board, 'w')
                                            if is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                                checkmate = True
                                            else:
                                                stalemate = True
                                        if best_row == best_col == target_row == target_col == best_piece == captured == '2':
                                            bad_checkmate = True
                                        if not checkmate and not bad_checkmate and not stalemate:
                                            if best_piece != 'P':
                                              if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                              else:
                                                print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                            else:
                                              if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                              else:
                                                print(indices_to_pos(target_row, target_col))
                                            board[best_row][best_col] = '0'
                                            board[target_row][target_col] = best_piece
                                            if target_row == 7 and best_piece == 'P':
                                                board[target_row][target_col] = 'Q'
                                            best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                            if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                                checkmate2 = True
                                            if not checkmate2:
                                                if best_piece2 != 'p':
                                                  if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                    print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                                  else:
                                                    print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                                else:
                                                  if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                    print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                                  else:
                                                    print(indices_to_pos(target_row2, target_col2))
                                                board[best_row2][best_col2] = '0'
                                                board[target_row2][target_col2] = best_piece2
                                                if target_row2 == 0 and best_piece2 == 'p':
                                                    board[target_row2][target_col2] = 'q'
                                                current_score = score(board, 'w')
                                                for move in good_moves:
                                                    if (row, col, new_row, new_col, 'r', captured_piece) == move:
                                                        current_score += 0.5
                                                print(current_score)
                                                print()
                                                board[best_row2][best_col2] = best_piece2
                                                board[target_row2][target_col2] = captured2
                                                board[best_row][best_col] = best_piece
                                                board[target_row][target_col] = captured
                                                if current_score > previous_score:
                                                    previous_score = current_score
                                                    best_moves = [(row, col, new_row, new_col, 'r')]
                                                elif current_score == previous_score:
                                                    best_moves.append((row, col, new_row, new_col, 'r'))
                                        elif checkmate:
                                            print_board(board)
                                            print('CHECKMATE! you lose')
                                            sys.exit()
                                        elif bad_checkmate:
                                            current_score = -1000
                                            if current_score > previous_score:
                                                previous_score = current_score
                                                best_moves = [(row, col, new_row, new_col, 'r')]
                                            elif current_score == previous_score:
                                                best_moves.append((row, col, new_row, new_col, 'r'))
                                    board[row][col] = 'r'
                                    board[new_row][new_col] = captured_piece
                                    checkmate = False
                                    checkmate2 = False
                                    bad_checkmate = False
                                    stalemate = False

                                else:
                                    break
                                if board[new_row][new_col] == 'P' or board[new_row][new_col] == 'N' or board[new_row][new_col] == 'B' or board[new_row][new_col] == 'R' or board[new_row][new_col] == 'Q':
                                    break
                            else:
                                break

                elif piece == 'q':
                    directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
                    random.shuffle(directions)
                    for direction in directions:
                        for i in range(1, 8):
                            new_row = row + i * direction[0]
                            new_col = col + i * direction[1]
                            if 0 <= new_row < 8 and 0 <= new_col < 8:
                                if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                    captured_piece = board[new_row][new_col]
                                    if board[new_row][new_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                      print('Q' + 'x' + indices_to_pos(new_row, new_col))
                                    else:
                                      print('Q' + indices_to_pos(new_row, new_col))
                                    board[row][col] = '0'
                                    board[new_row][new_col] = 'q'
                                    if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                        best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                        if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                            white_king_row, white_king_col = find_king(board, 'w')
                                            if is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                                checkmate = True
                                            else:
                                                stalemate = True
                                        if best_row == best_col == target_row == target_col == best_piece == captured == '2':
                                            bad_checkmate = True
                                        if not checkmate and not bad_checkmate and not stalemate:
                                            if best_piece != 'P':
                                              if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                              else:
                                                print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                            else:
                                              if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                              else:
                                                print(indices_to_pos(target_row, target_col))
                                            board[best_row][best_col] = '0'
                                            board[target_row][target_col] = best_piece
                                            if target_row == 7 and best_piece == 'P':
                                                board[target_row][target_col] = 'Q'
                                            best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                            if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                                checkmate2 = True
                                            if not checkmate2:
                                                if best_piece2 != 'p':
                                                  if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                    print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                                  else:
                                                    print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                                else:
                                                  if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                    print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                                  else:
                                                    print(indices_to_pos(target_row2, target_col2))
                                                board[best_row2][best_col2] = '0'
                                                board[target_row2][target_col2] = best_piece2
                                                if target_row2 == 0 and best_piece2 == 'p':
                                                    board[target_row2][target_col2] = 'q'
                                                current_score = score(board, 'w')
                                                for move in good_moves:
                                                    if (row, col, new_row, new_col, 'q', captured_piece) == move:
                                                        current_score += 0.5
                                                print(current_score)
                                                print()
                                                board[best_row2][best_col2] = best_piece2
                                                board[target_row2][target_col2] = captured2
                                                board[best_row][best_col] = best_piece
                                                board[target_row][target_col] = captured
                                                if current_score > previous_score:
                                                    previous_score = current_score
                                                    best_moves = [(row, col, new_row, new_col, 'q')]
                                                elif current_score == previous_score:
                                                    best_moves.append((row, col, new_row, new_col, 'q'))
                                        elif checkmate:
                                            print_board(board)
                                            print('CHECKMATE! you lose')
                                            sys.exit()
                                        elif bad_checkmate:
                                            current_score = -1000
                                            if current_score > previous_score:
                                                previous_score = current_score
                                                best_moves = [(row, col, new_row, new_col, 'q')]
                                            elif current_score == previous_score:
                                                best_moves.append((row, col, new_row, new_col, 'q'))
                                    board[row][col] = 'q'
                                    board[new_row][new_col] = captured_piece
                                    checkmate = False
                                    checkmate2 = False
                                    bad_checkmate = False
                                    stalemate = False

                                else:
                                    break
                                if board[new_row][new_col] == 'P' or board[new_row][new_col] == 'N' or board[new_row][new_col] == 'B' or board[new_row][new_col] == 'R' or board[new_row][new_col] == 'Q':
                                    break
                            else:
                                break

                elif piece == 'k':
                    directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1), '0-0', '0-0-0']
                    random.shuffle(directions)
                    for direction in directions:
                        if direction not in {'0-0', '0-0-0'}:
                            new_row = row + direction[0]
                            new_col = col + direction[1]
                            if 0 <= new_row < 8 and 0 <= new_col < 8:
                                if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                    captured_piece = board[new_row][new_col]
                                    if board[new_row][new_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                      print('K' + 'x' + indices_to_pos(new_row, new_col))
                                    else:
                                      print('K' + indices_to_pos(new_row, new_col))
                                    board[row][col] = '0'
                                    board[new_row][new_col] = 'k'
                                    black_king_row, black_king_col = find_king(board, 'b')
                                    if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                        best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                        if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                            white_king_row, white_king_col = find_king(board, 'w')
                                            if is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                                checkmate = True
                                            else:
                                                stalemate = True
                                        if best_row == best_col == target_row == target_col == best_piece == captured == '2':
                                            bad_checkmate = True
                                        if not checkmate and not bad_checkmate and not stalemate:
                                            if best_piece != 'P':
                                              if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                              else:
                                                print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                            else:
                                                if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                  print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                                else:
                                                  print(indices_to_pos(target_row, target_col))
                                            board[best_row][best_col] = '0'
                                            board[target_row][target_col] = best_piece
                                            if target_row == 7 and best_piece == 'P':
                                                board[target_row][target_col] = 'Q'
                                            best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                            if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                                checkmate2 = True
                                            if not checkmate2:
                                                if best_piece2 != 'p':
                                                    if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                      print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                                    else:
                                                      print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                                else:
                                                    if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                      print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                                    else:
                                                      print(indices_to_pos(target_row2, target_col2))
                                                board[best_row2][best_col2] = '0'
                                                board[target_row2][target_col2] = best_piece2
                                                if target_row2 == 0 and best_piece2 == 'p':
                                                    board[target_row2][target_col2] = 'q'
                                                current_score = score(board, 'w')
                                                for move in good_moves:
                                                    if (row, col, new_row, new_col, 'k', captured_piece) == move:
                                                        current_score += 0.5
                                                print(current_score)
                                                print()
                                                board[best_row2][best_col2] = best_piece2
                                                board[target_row2][target_col2] = captured2
                                                board[best_row][best_col] = best_piece
                                                board[target_row][target_col] = captured
                                                if current_score > previous_score:
                                                    previous_score = current_score
                                                    best_moves = [(row, col, new_row, new_col, 'k')]
                                                elif current_score == previous_score:
                                                    best_moves.append((row, col, new_row, new_col, 'k'))
                                        elif checkmate:
                                            print_board(board)
                                            print('CHECKMATE! you lose')
                                            sys.exit()
                                        elif bad_checkmate:
                                            current_score = -1000
                                            if current_score > previous_score:
                                                previous_score = current_score
                                                best_moves = [(row, col, new_row, new_col, 'k')]
                                            elif current_score == previous_score:
                                                best_moves.append((row, col, new_row, new_col, 'k'))
                                    else:
                                        print('In check')
                                    board[row][col] = 'k'
                                    board[new_row][new_col] = captured_piece
                                    checkmate = False
                                    checkmate2 = False
                                    bad_checkmate = False
                                    stalemate = False
                                    black_king_row, black_king_col = find_king(board, 'b')
                        else:
                            if direction == '0-0':
                                if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                    if board[7][4] == 'k' and board[7][7] == 'r' and king_move == 0 and board[7][5] == '0' and board[7][6] == '0':
                                        board[7][5] = 'k'
                                        board[7][4] = '0'
                                        if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                            board[7][6] = 'k'
                                            board[7][5] = '0'
                                            if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                                print('0-0')
                                                board[7][7] = '0'
                                                board[7][5] = 'r'
                                                best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                                if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                                    checkmate = True
                                                if best_row == best_col == target_row == target_col == best_piece == captured == 2:
                                                    bad_checkmate = True
                                                if not checkmate and not bad_checkmate:
                                                    if best_piece != 'P':
                                                      if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                        print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                                      else:
                                                        print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                                    else:
                                                        if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                          print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                                        else:
                                                          print(indices_to_pos(target_row, target_col))
                                                    board[best_row][best_col] = '0'
                                                    board[target_row][target_col] = best_piece
                                                    best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                                    if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                                        checkmate2 = True
                                                    if not checkmate2:
                                                        if best_piece2 != 'p':
                                                            if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                              print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                                            else:
                                                              print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                                        else:
                                                            if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                              print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                                            else:
                                                              print(indices_to_pos(target_row2, target_col2))
                                                        board[best_row2][best_col2] = '0'
                                                        board[target_row2][target_col2] = best_piece2
                                                        current_score = score(board, 'w')
                                                        print(current_score)
                                                        print()
                                                        print_board(board)
                                                        print()
                                                        board[best_row2][best_col2] = best_piece2
                                                        board[target_row2][target_col2] = captured2
                                                        board[best_row][best_col] = best_piece
                                                        board[target_row][target_col] = captured
                                                        if current_score > previous_score:
                                                            previous_score = current_score
                                                            best_moves = [('0-0')]
                                                        elif current_score == previous_score:
                                                            best_moves.append(('0-0'))
                                                        board[7][6] = '0'
                                                        board[7][4] = 'k'
                                                        board[7][6] = '0'
                                                        board[7][7] = 'r'
                                                        board[7][5] = '0'
                                                        checkmate = False
                                                        checkmate2 = False
                                                        black_king_row, black_king_col = find_king(board, 'b')
                                                elif checkmate:
                                                        print_board(board)
                                                        print('CHECKMATE! you lose')
                                            else:
                                                board[7][4] = 'k'
                                                board[7][6] = '0'
                                        else:
                                            board[7][4] = 'k'
                                            board[7][5] = '0'

                            elif direction == '0-0-0':
                                if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                    if board[7][4] == 'k' and board[7][0] == 'r' and king_move == 0 and board[7][1] == '0' and board[7][2] == '0' and board[7][3] == '0':
                                        board[7][3] = 'k'
                                        board[7][4] = '0'
                                        if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                            board[7][2] = 'k'
                                            board[7][3] = '0'
                                            if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                                print('0-0-0')
                                                board[7][0] = '0'
                                                board[7][3] = 'r'
                                                best_row, best_col, target_row, target_col, best_piece, captured = best_move_player(board)
                                                if best_row == best_col == target_row == target_col == best_piece == captured == 1:
                                                    checkmate = True
                                                if best_row == best_col == target_row == target_col == best_piece == captured == 2:
                                                    bad_checkmate = True
                                                if not checkmate and not bad_checkmate:
                                                    if best_piece != 'P':
                                                      if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                        print(best_piece.upper() + 'x' + indices_to_pos(target_row, target_col))
                                                      else:
                                                        print(best_piece.upper() + indices_to_pos(target_row, target_col))
                                                    else:
                                                        if board[target_row][target_col] in {'p', 'n', 'b', 'r', 'q'}:
                                                          print(indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col))
                                                        else:
                                                          print(indices_to_pos(target_row, target_col))
                                                    board[best_row][best_col] = '0'
                                                    board[target_row][target_col] = best_piece
                                                    best_row2, best_col2, target_row2, target_col2, best_piece2, captured2 = best_move2(board)
                                                    if best_row2 == best_col2 == target_row2 == target_col2 == best_piece2 == captured2 == '1':
                                                        checkmate2 = True
                                                    if not checkmate2:
                                                        if best_piece2 != 'p':
                                                            if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                              print(best_piece2.upper() + 'x' + indices_to_pos(target_row2, target_col2))
                                                            else:
                                                              print(best_piece2.upper() + indices_to_pos(target_row2, target_col2))
                                                        else:
                                                            if board[target_row2][target_col2] in {'P', 'N', 'B', 'R', 'Q'}:
                                                              print(indices_to_pos_col(best_col2) + 'x' + indices_to_pos(target_row2, target_col2))
                                                            else:
                                                              print(indices_to_pos(target_row2, target_col2))
                                                        board[best_row2][best_col2] = '0'
                                                        board[target_row2][target_col2] = best_piece2
                                                        current_score = score(board, 'w')
                                                        print(current_score)
                                                        print()
                                                        print_board(board)
                                                        print()
                                                        board[best_row2][best_col2] = best_piece2
                                                        board[target_row2][target_col2] = captured2
                                                        board[best_row][best_col] = best_piece
                                                        board[target_row][target_col] = captured
                                                        if current_score > previous_score:
                                                            previous_score = current_score
                                                            best_moves = [('0-0-0')]
                                                        elif current_score == previous_score:
                                                            best_moves.append(('0-0-0'))
                                                        board[7][4] = 'k'
                                                        board[7][2] = '0'
                                                        board[7][0] = 'r'
                                                        board[7][3] = '0'
                                                        checkmate = False
                                                        checkmate2 = False
                                                        black_king_row, black_king_col = find_king(board, 'b')
                                                elif checkmate:
                                                    print_board(board)
                                                    print('CHECKMATE! you lose')
                                            else:
                                                board[7][4] = 'k'
                                                board[7][2] = '0'
                                        else:
                                            board[7][4] = 'k'
                                            board[7][3] = '0'

    if best_moves:
        best_move = random.choice(best_moves)
        if best_move != '0-0' and best_move != '0-0-0':
            best_row, best_col, target_row, target_col, best_piece = best_move
            captured_piece = board[target_row][target_col]
            piece = board[target_row][target_col]
            board[best_row][best_col] = '0'
            board[target_row][target_col] = best_piece
            print_board(board)
            print()
            if best_piece == 'k':
                king_move = 1
            white_king_row, white_king_col = find_king(board, 'w')
            if is_king_in_check(board, white_king_row, white_king_col, 'w'):
                if best_piece != 'p':
                      if piece in {'P', 'N', 'B', 'R', 'Q'}:
                        move_played = best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + '+'
                        print(move_played)
                      else:
                        move_played = best_piece.upper() + indices_to_pos(target_row, target_col) + '+'
                        print(move_played)
                else:
                      if piece in {'P', 'N', 'B', 'R', 'Q'}:
                        move_played = indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col + '+')
                        print(move_played)
                      else:
                        move_played = indices_to_pos(target_row, target_col + '+')
                        print(move_played)
            else:
                if best_piece != 'p':
                      if piece in {'P', 'N', 'B', 'R', 'Q'}:
                        move_played = best_piece.upper() + 'x' + indices_to_pos(target_row, target_col)
                        print(move_played)
                      else:
                        move_played = best_piece.upper() + indices_to_pos(target_row, target_col)
                        print(move_played)
                else:
                      if piece in {'P', 'N', 'B', 'R', 'Q'}:
                        move_played = indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col)
                        print(move_played)
                      else:
                        move_played = indices_to_pos(target_row, target_col)
                        print(move_played)
        elif best_move == '0-0':
            move_played = '0-0'
            king_move = 1
            board[7][4] = '0'
            board[7][7] = '0'
            board[7][6] = 'k'
            board[7][5] = 'r'
            print_board(board)
            print()
            print('0-0')
        elif best_move == '0-0-0':
            king_move = 1
            move_played = '0-0-0'
            board[7][4] = '0'
            board[7][0] = '0'
            board[7][2] = 'k'
            board[7][3] = 'r'
            print_board(board)
            print()
            print('0-0-0')
        print(previous_score)
        print()
        game_moves.append(move_played)
        number_of_moves += 1
        move_number = 0
        for number in range(number_of_moves):
            print(str(number+1) + '. ' + game_moves[move_number], end=' ')
            if number != number_of_moves-1:
                move_number += 1
                print(game_moves[move_number], end=' ')
                move_number += 1

        stop_timer()
        players_turn(board)


def best_move_player(board):
    white_king_row, white_king_col = find_king(board, 'w')
    checkmate = False
    promotion = False
    previous_score = 60
    best_moves = []
    rows = list(range(8))
    random.shuffle(rows)
    cols = list(range(8))
    random.shuffle(cols)

    for row in rows:
        for col in cols:
            piece = board[row][col]
            if piece == 'P':
                directions = [1, 2, 3, 4]
                random.shuffle(directions)
                for direction in directions:
                    if direction == 1 and row == 1 and board[row+2][col] == '0' and board[row+1][col] == '0':
                        board[row][col] = '0'
                        board[row+2][col] = 'P'
                        if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                            best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                            if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                checkmate = True
                                print('hahaha')
                            if not checkmate:
                                board[best_row][best_col] = '0'
                                board[target_row][target_col] = best_piece
                                if target_row == 0 and best_piece == 'p':
                                    board[target_row][target_col] = 'q'
                                current_score = score(board, 'w')
                                board[best_row][best_col] = best_piece
                                board[target_row][target_col] = captured
                                if current_score < previous_score:
                                    previous_score = current_score
                                    best_moves = [(row, col, row+2, col, 'P')]
                                elif current_score == previous_score:
                                    best_moves.append((row, col, row+2, col, 'P'))
                                    if best_piece != 'p':
                                      if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                        analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                      else:
                                        analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                    else:
                                        if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                          analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                        else:
                                          analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                            else:
                                board[row][col] = 'P'
                                board[row+2][col] = '0'
                                checkmate = False
                                return '2', '2', '2', '2', '2', '2'
                        board[row][col] = 'P'
                        board[row+2][col] = '0'
                        checkmate = False

                    elif direction == 2 and row < 7 and board[row+1][col] == '0':
                        board[row][col] = '0'
                        board[row+1][col] = 'P'
                        if row+1 == 7:
                            board[row+1][col] = 'Q'
                            promotion = True
                        if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                            best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                            if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                checkmate = True
                                print('hahaha2')
                            if not checkmate:
                                board[best_row][best_col] = '0'
                                board[target_row][target_col] = best_piece
                                if target_row == 0 and best_piece == 'p':
                                    board[target_row][target_col] = 'q'
                                current_score = score(board, 'w')
                                board[best_row][best_col] = best_piece
                                board[target_row][target_col] = captured
                                if current_score < previous_score:
                                    previous_score = current_score
                                    best_moves = [(row, col, row+1, col, 'P')]
                                    if best_piece != 'p':
                                      if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                        analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                      else:
                                        analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                    else:
                                        if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                          analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                        else:
                                          analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                                elif current_score == previous_score:
                                    best_moves.append((row, col, row+1, col, 'P'))
                            else:
                                board[row][col] = 'P'
                                board[row+1][col] = '0'
                                checkmate = False
                                return '2', '2', '2', '2', '2', '2'
                        board[row][col] = 'P'
                        board[row+1][col] = '0'
                        checkmate = False

                    elif direction == 3 and row < 7 and col > 0 and board[row+1][col-1] in {'p', 'n', 'b', 'r', 'q'}:
                        captured_piece = board[row+1][col-1]
                        board[row][col] = '0'
                        board[row+1][col-1] = 'P'
                        if row+1 == 7:
                            board[row+1][col-1] = 'Q'
                            promotion = True
                        if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                            best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                            if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                checkmate = True
                                print('hahaha3')
                            if not checkmate:
                                board[best_row][best_col] = '0'
                                board[target_row][target_col] = best_piece
                                if target_row == 0 and best_piece == 'p':
                                    board[target_row][target_col] = 'q'
                                current_score = score(board, 'w')
                                board[best_row][best_col] = best_piece
                                board[target_row][target_col] = captured
                                if current_score < previous_score:
                                    previous_score = current_score
                                    best_moves = [(row, col, row+1, col-1, 'P')]
                                    if best_piece != 'p':
                                      if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                        analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                      else:
                                        analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                    else:
                                        if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                          analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                        else:
                                          analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                                elif current_score == previous_score:
                                    best_moves.append((row, col, row+1, col-1, 'P'))
                            else:
                                board[row][col] = 'P'
                                board[row+1][col-1] = captured_piece
                                checkmate = False
                                return '2', '2', '2', '2', '2', '2'
                        board[row][col] = 'P'
                        board[row+1][col-1] = captured_piece
                        checkmate = False

                    elif direction == 4 and row > 0 and col < 7 and board[row+1][col+1] in {'p', 'n', 'b', 'r', 'q'}:
                        captured_piece = board[row+1][col+1]
                        board[row][col] = '0'
                        board[row+1][col+1] = 'P'
                        if row+1 == 7:
                            board[row+1][col+1] = 'Q'
                            promotion = True
                        if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                            best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                            if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                checkmate = True
                                print('hahaha4')
                            if not checkmate:
                                board[best_row][best_col] = '0'
                                board[target_row][target_col] = best_piece
                                if target_row == 0 and best_piece == 'p':
                                    board[target_row][target_col] = 'q'
                                current_score = score(board, 'w')
                                board[best_row][best_col] = best_piece
                                board[target_row][target_col] = captured
                                if current_score < previous_score:
                                    previous_score = current_score
                                    best_moves = [(row, col, row+1, col+1, 'P')]
                                    if best_piece != 'p':
                                      if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                        analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                      else:
                                        analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                    else:
                                        if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                          analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                        else:
                                          analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                                elif current_score == previous_score:
                                    best_moves.append((row, col, row+1, col+1, 'P'))
                            else:
                                board[row][col] = 'P'
                                board[row+1][col+1] = captured_piece
                                checkmate = False
                                return '2', '2', '2', '2', '2', '2'
                        board[row][col] = 'P'
                        board[row+1][col+1] = captured_piece
                        checkmate = False

            elif piece == 'N':
                directions = [(1, 2), (1, -2), (-1, 2), (-1, -2), (2, 1), (2, -1), (-2, 1), (-2, -1)]
                random.shuffle(directions)
                for direction in directions:
                    new_row = row + direction[0]
                    new_col = col + direction[1]
                    if 0 <= new_row < 8 and 0 <= new_col < 8:
                        if board[new_row][new_col] in {'0', 'p', 'n', 'b', 'r', 'q'}:
                            captured_piece = board[new_row][new_col]
                            board[row][col] = '0'
                            board[new_row][new_col] = 'N'
                            if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                                if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                    checkmate = True
                                    print('hahaha N')
                                if not checkmate:
                                    board[best_row][best_col] = '0'
                                    board[target_row][target_col] = best_piece
                                    if target_row == 0 and best_piece == 'p':
                                        board[target_row][target_col] = 'q'
                                    current_score = score(board, 'w')
                                    board[best_row][best_col] = best_piece
                                    board[target_row][target_col] = captured
                                    if current_score < previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, new_row, new_col, 'N')]
                                        if best_piece != 'p':
                                          if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                            analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                          else:
                                            analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                        else:
                                            if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                              analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                            else:
                                              analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, new_row, new_col, 'N'))
                                else:
                                    board[row][col] = 'N'
                                    board[new_row][new_col] = captured_piece
                                    checkmate = False
                                    return '2', '2', '2', '2', '2', '2'
                            board[row][col] = 'N'
                            board[new_row][new_col] = captured_piece
                            checkmate = False

            elif piece == 'B':
                directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
                random.shuffle(directions)
                for direction in directions:
                    for i in range(1, 8):
                        new_row = row + i * direction[0]
                        new_col = col + i * direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'p', 'n', 'b', 'r', 'q'}:
                                captured_piece = board[new_row][new_col]
                                board[row][col] = '0'
                                board[new_row][new_col] = 'B'
                                if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                    best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                                    if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                        checkmate = True
                                        print('hahaha B')
                                    if not checkmate:
                                        board[best_row][best_col] = '0'
                                        board[target_row][target_col] = best_piece
                                        if target_row == 0 and best_piece == 'p':
                                            board[target_row][target_col] = 'q'
                                        current_score = score(board, 'w')
                                        board[best_row][best_col] = best_piece
                                        board[target_row][target_col] = captured
                                        if current_score < previous_score:
                                            previous_score = current_score
                                            best_moves = [(row, col, new_row, new_col, 'B')]
                                            if best_piece != 'p':
                                              if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                                analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                              else:
                                                analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                            else:
                                                if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                                  analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                                else:
                                                  analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                                        elif current_score == previous_score:
                                            best_moves.append((row, col, new_row, new_col, 'B'))
                                    else:
                                        board[row][col] = 'B'
                                        board[new_row][new_col] = captured_piece
                                        checkmate = False
                                        return '2', '2', '2', '2', '2', '2'
                                board[row][col] = 'B'
                                board[new_row][new_col] = captured_piece
                                checkmate = False
                            else:
                                break
                            if board[new_row][new_col] == 'p' or board[new_row][new_col] == 'n' or board[new_row][new_col] == 'b' or board[new_row][new_col] == 'r' or board[new_row][new_col] == 'q':
                                break
                        else:
                            break

            elif piece == 'R':
                directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
                random.shuffle(directions)
                for direction in directions:
                    for i in range(1, 8):
                        new_row = row + i * direction[0]
                        new_col = col + i * direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'p', 'n', 'b', 'r', 'q'}:
                                captured_piece = board[new_row][new_col]
                                board[row][col] = '0'
                                board[new_row][new_col] = 'R'
                                if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                    best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                                    if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                        checkmate = True
                                        print('hahaha R')
                                    if not checkmate:
                                        if best_piece != 'p':
                                          if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                            analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                          else:
                                            analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                        else:
                                            if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                              analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                            else:
                                              analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                                        board[best_row][best_col] = '0'
                                        board[target_row][target_col] = best_piece
                                        if target_row == 0 and best_piece == 'p':
                                            board[target_row][target_col] = 'q'
                                        current_score = score(board, 'w')
                                        board[best_row][best_col] = best_piece
                                        board[target_row][target_col] = captured
                                        if current_score < previous_score:
                                            previous_score = current_score
                                            best_moves = [(row, col, new_row, new_col, 'R')]
                                            if best_piece != 'p':
                                              if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                                analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                              else:
                                                analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                            else:
                                                if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                                  analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                                else:
                                                  analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                                        elif current_score == previous_score:
                                            best_moves.append((row, col, new_row, new_col, 'R'))
                                    else:
                                        board[row][col] = 'R'
                                        board[new_row][new_col] = captured_piece
                                        checkmate = False
                                        return '2', '2', '2', '2', '2', '2'
                                board[row][col] = 'R'
                                board[new_row][new_col] = captured_piece
                                checkmate = False
                            else:
                                break
                            if board[new_row][new_col] == 'p' or board[new_row][new_col] == 'n' or board[new_row][new_col] == 'b' or board[new_row][new_col] == 'r' or board[new_row][new_col] == 'q':
                                break
                        else:
                            break

            elif piece == 'Q':
                directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
                random.shuffle(directions)
                for direction in directions:
                    for i in range(1, 8):
                        new_row = row + i * direction[0]
                        new_col = col + i * direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'p', 'n', 'b', 'r', 'q'}:
                                captured_piece = board[new_row][new_col]
                                board[row][col] = '0'
                                board[new_row][new_col] = 'Q'
                                if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                    best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                                    if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                        checkmate = True
                                        print('hahaha Q')
                                    if not checkmate:
                                        board[best_row][best_col] = '0'
                                        board[target_row][target_col] = best_piece
                                        if target_row == 0 and best_piece == 'p':
                                            board[target_row][target_col] = 'q'
                                        current_score = score(board, 'w')
                                        board[best_row][best_col] = best_piece
                                        board[target_row][target_col] = captured
                                        if current_score < previous_score:
                                            previous_score = current_score
                                            best_moves = [(row, col, new_row, new_col, 'Q')]
                                            if best_piece != 'p':
                                              if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                                analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                              else:
                                                analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                            else:
                                                if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                                  analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                                else:
                                                  analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                                        elif current_score == previous_score:
                                            best_moves.append((row, col, new_row, new_col, 'Q'))
                                    else:
                                        board[row][col] = 'Q'
                                        board[new_row][new_col] = captured_piece
                                        checkmate = False
                                        return '2', '2', '2', '2', '2', '2'
                                board[row][col] = 'Q'
                                board[new_row][new_col] = captured_piece
                                checkmate = False
                            else:
                                break
                            if board[new_row][new_col] == 'p' or board[new_row][new_col] == 'n' or board[new_row][new_col] == 'b' or board[new_row][new_col] == 'r' or board[new_row][new_col] == 'q':
                                break
                        else:
                            break

            elif piece == 'K':
                directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
                random.shuffle(directions)
                for direction in directions:
                    new_row = row + direction[0]
                    new_col = col + direction[1]
                    if 0 <= new_row < 8 and 0 <= new_col < 8:
                        if board[new_row][new_col] in {'0', 'p', 'n', 'b', 'r', 'q'}:
                            captured_piece = board[new_row][new_col]
                            board[row][col] = '0'
                            board[new_row][new_col] = 'K'
                            white_king_row, white_king_col = find_king(board, 'w')
                            if not is_king_in_check(board, white_king_row, white_king_col, 'w'):
                                best_row, best_col, target_row, target_col, best_piece, captured = best_move2(board)
                                if best_row == best_col == target_row == target_col == best_piece == captured == '1':
                                    checkmate = True
                                    print('hahaha K')
                                if not checkmate:
                                    board[best_row][best_col] = '0'
                                    board[target_row][target_col] = best_piece
                                    if target_row == 0 and best_piece == 'p':
                                        board[target_row][target_col] = 'q'
                                    current_score = score(board, 'w')
                                    board[best_row][best_col] = best_piece
                                    board[target_row][target_col] = captured
                                    if current_score < previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, new_row, new_col, 'K')]
                                        if best_piece != 'p':
                                          if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                            analized = ('(' + best_piece.upper() + 'x' + indices_to_pos(target_row, target_col) + ')')
                                          else:
                                            analized = ('(' + best_piece.upper() + indices_to_pos(target_row, target_col) + ')')
                                        else:
                                            if board[target_row][target_col] in {'P', 'N', 'B', 'R', 'Q'}:
                                              analized = ('(' + indices_to_pos_col(best_col) + 'x' + indices_to_pos(target_row, target_col) + ')')
                                            else:
                                              analized = ('(' + indices_to_pos(target_row, target_col) + ')')
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, new_row, new_col, 'K'))
                                else:
                                    board[row][col] = 'K'
                                    board[new_row][new_col] = captured_piece
                                    checkmate = False
                                    white_king_row, white_king_col = find_king(board, 'w')
                                    return '2', '2', '2', '2', '2', '2'
                            board[row][col] = 'K'
                            board[new_row][new_col] = captured_piece
                            checkmate = False
                            white_king_row, white_king_col = find_king(board, 'w')

    if best_moves:
        best_move = random.choice(best_moves)
        best_row, best_col, target_row, target_col, best_piece = best_move
        captured_piece = board[target_row][target_col]
        board[best_row][best_col] = '0'
        board[target_row][target_col] = best_piece
        return best_row, best_col, target_row, target_col, best_piece, captured_piece
    else:
        return '1', '1', '1', '1', '1', '1'

def best_move2(board):
    black_king_row, black_king_col = find_king(board, 'b')
    promotion = False
    previous_score = -60
    best_moves = []
    rows = list(range(8))
    random.shuffle(rows)
    cols = list(range(8))
    random.shuffle(cols)

    for row in rows:
        for col in cols:
            piece = board[row][col]
            if piece == 'p':
                directions = [1, 2, 3, 4]
                random.shuffle(directions)
                for direction in directions:
                    if direction == 1 and row == 1 and board[row-2][col] == '0' and board[row-1][col] == '0':
                        board[row][col] = '0'
                        board[row-2][col] = 'p'
                        if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                            current_score = score(board, 'w')
                            if current_score > previous_score:
                                previous_score = current_score
                                best_moves = [(row, col, row-2, col, 'p')]
                            elif current_score == previous_score:
                                best_moves.append((row, col, row-2, col, 'p'))
                        board[row][col] = 'p'
                        board[row-2][col] = '0'

                    elif direction == 2 and row < 7 and board[row-1][col] == '0':
                        board[row][col] = '0'
                        board[row-1][col] = 'p'
                        if row-1 == 0:
                            board[row-1][col] = 'q'
                        if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                            current_score = score(board, 'w')
                            if current_score > previous_score:
                                previous_score = current_score
                                best_moves = [(row, col, row-1, col, 'p')]
                            elif current_score == previous_score:
                                best_moves.append((row, col, row-1, col, 'p'))
                        board[row][col] = 'p'
                        board[row-1][col] = '0'

                    elif direction == 3 and row < 7 and col > 0 and board[row-1][col-1] in {'P', 'N', 'B', 'R', 'Q'}:
                        captured_piece = board[row-1][col-1]
                        board[row][col] = '0'
                        board[row-1][col-1] = 'p'
                        if row-1 == 0:
                            board[row-1][col-1] = 'q'
                        if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                            current_score = score(board, 'w')
                            if current_score > previous_score:
                                previous_score = current_score
                                best_moves = [(row, col, row-1, col-1, 'p')]
                            elif current_score == previous_score:
                                best_moves.append((row, col, row-1, col-1, 'p'))
                        board[row][col] = 'p'
                        board[row-1][col-1] = captured_piece

                    elif direction == 4 and row > 0 and col < 7 and board[row-1][col+1] in {'P', 'N', 'B', 'R', 'Q'}:
                        captured_piece = board[row-1][col+1]
                        board[row][col] = '0'
                        board[row-1][col+1] = 'p'
                        if row-1 == 0:
                            board[row-1][col+1] = 'q'
                            promotion = True
                        if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                            current_score = score(board, 'w')
                            if current_score > previous_score:
                                previous_score = current_score
                                best_moves = [(row, col, row-1, col+1, 'p')]
                            elif current_score == previous_score:
                                best_moves.append((row, col, row-1, col+1, 'p'))
                        board[row][col] = 'p'
                        board[row-1][col+1] = captured_piece

            elif piece == 'n':
                directions = [(1, 2), (1, -2), (-1, 2), (-1, -2), (2, 1), (2, -1), (-2, 1), (-2, -1)]
                random.shuffle(directions)
                for direction in directions:
                    new_row = row + direction[0]
                    new_col = col + direction[1]
                    if 0 <= new_row < 8 and 0 <= new_col < 8:
                        if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                            captured_piece = board[new_row][new_col]
                            board[row][col] = '0'
                            board[new_row][new_col] = 'n'
                            if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                current_score = score(board, 'w')
                                if current_score > previous_score:
                                    previous_score = current_score
                                    best_moves = [(row, col, new_row, new_col, 'n')]
                                elif current_score == previous_score:
                                    best_moves.append((row, col, new_row, new_col, 'n'))
                            board[row][col] = 'n'
                            board[new_row][new_col] = captured_piece

            elif piece == 'b':
                directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
                random.shuffle(directions)
                for direction in directions:
                    for i in range(1, 8):
                        new_row = row + i * direction[0]
                        new_col = col + i * direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                captured_piece = board[new_row][new_col]
                                board[row][col] = '0'
                                board[new_row][new_col] = 'b'
                                if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                    current_score = score(board, 'w')
                                    if current_score > previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, new_row, new_col, 'b')]
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, new_row, new_col, 'b'))
                                board[row][col] = 'b'
                                board[new_row][new_col] = captured_piece
                            else:
                                break
                            if board[new_row][new_col] == 'P' or board[new_row][new_col] == 'N' or board[new_row][new_col] == 'B' or board[new_row][new_col] == 'R' or board[new_row][new_col] == 'Q':
                                break
                        else:
                            break

            elif piece == 'r':
                directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
                random.shuffle(directions)
                for direction in directions:
                    for i in range(1, 8):
                        new_row = row + i * direction[0]
                        new_col = col + i * direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                captured_piece = board[new_row][new_col]
                                board[row][col] = '0'
                                board[new_row][new_col] = 'r'
                                if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                    current_score = score(board, 'w')
                                    if current_score > previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, new_row, new_col, 'r')]
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, new_row, new_col, 'r'))
                                board[row][col] = 'r'
                                board[new_row][new_col] = captured_piece
                            else:
                                break
                            if board[new_row][new_col] == 'P' or board[new_row][new_col] == 'N' or board[new_row][new_col] == 'B' or board[new_row][new_col] == 'R' or board[new_row][new_col] == 'Q':
                                break
                        else:
                            break

            elif piece == 'q':
                directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
                random.shuffle(directions)
                for direction in directions:
                    for i in range(1, 8):
                        new_row = row + i * direction[0]
                        new_col = col + i * direction[1]
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                                captured_piece = board[new_row][new_col]
                                board[row][col] = '0'
                                board[new_row][new_col] = 'q'
                                if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                    current_score = score(board, 'w')
                                    if current_score > previous_score:
                                        previous_score = current_score
                                        best_moves = [(row, col, new_row, new_col, 'q')]
                                    elif current_score == previous_score:
                                        best_moves.append((row, col, new_row, new_col, 'q'))
                                board[row][col] = 'q'
                                board[new_row][new_col] = captured_piece
                            else:
                                break
                            if board[new_row][new_col] == 'P' or board[new_row][new_col] == 'N' or board[new_row][new_col] == 'B' or board[new_row][new_col] == 'R' or board[new_row][new_col] == 'Q':
                                break
                        else:
                            break

            elif piece == 'k':
                directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
                random.shuffle(directions)
                for direction in directions:
                    new_row = row + direction[0]
                    new_col = col + direction[1]
                    if 0 <= new_row < 8 and 0 <= new_col < 8:
                        if board[new_row][new_col] in {'0', 'P', 'N', 'B', 'R', 'Q'}:
                            captured_piece = board[new_row][new_col]
                            board[row][col] = '0'
                            board[new_row][new_col] = 'k'
                            black_king_row, black_king_col = find_king(board, 'b')
                            if not is_king_in_check(board, black_king_row, black_king_col, 'b'):
                                current_score = score(board, 'w')
                                if current_score > previous_score:
                                    previous_score = current_score
                                    best_moves = [(row, col, new_row, new_col, 'k')]
                                elif current_score == previous_score:
                                    best_moves.append((row, col, new_row, new_col, 'k'))
                            board[row][col] = 'k'
                            board[new_row][new_col] = captured_piece
                            black_king_row, black_king_col = find_king(board, 'b')

    if best_moves:
        best_move = random.choice(best_moves)
        best_row2, best_col2, target_row2, target_col2, best_piece2 = best_move
        captured_piece2 = board[target_row2][target_col2]
        board[best_row2][best_col2] = '0'
        board[target_row2][target_col2] = best_piece2
        return best_row2, best_col2, target_row2, target_col2, best_piece2, captured_piece2
    else:
        return '1', '1', '1', '1', '1', '1'

def train(board):
    good_moves = []
    while True:
        piece = input('What piece: ')
        from_square = input('From what square: ').strip().lower()
        to_square = input('To what square: ').strip().lower()
        captured_piece = input('Capturing what piece: ')
        from_row, from_col = pos_to_indices(from_square)
        to_row, to_col = pos_to_indices(to_square)
        good_moves.append((from_row, from_col, to_row, to_col, piece, captured_piece))
        if input('Continue or end? ') == 'end':
            break
    print(good_moves)

def game_loop():
    answer = input('Play or Train? ')
    if answer.lower() in {'t', 'train', 'not play'}:
        good_moves = train(board)
    else:
        start_timer()
        for _ in range(100):
            white_king_row, white_king_col = find_king(board, 'w')
            black_king_row, black_king_col = find_king(board, 'b')
            if is_king_in_check(board, white_king_row, white_king_col, 'w'):
                print("White king is in check!")
                players_turn(board)
            elif is_king_in_check(board, black_king_row, black_king_col, 'b'):
                print("Black king is in check!")
                best_move(board)
            else:
                best_move(board)

game_loop()
